# 네이버 영화평 감성분석

In [102]:
import numpy as np
import pandas as pd

In [103]:
#네이버 영와리뷰 데이터 로 검색하면 찾을수있음
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')

In [104]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [105]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

## 1. 데이터 전처리

### 트레인 데이터 셋

In [106]:
# 중복여부확인
train_df.document.nunique() # 안곂치는게 몇개인지

146182

In [107]:
# 중복데이터 삭제
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [108]:
# 위 둘의 숫자가 다르다 null 확인
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [109]:
#null 데이터 제거
train_df.dropna(how='any', inplace=True)
train_df.shape

(146182, 3)

In [110]:
# 긍정 부정 레이블의 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

### test data set 

In [111]:
# 중복여부확인
test_df.document.nunique() # 안곂치는게 몇개인지

49157

In [112]:
# 중복데이터 삭제
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [113]:
# 위 둘의 숫자가 다르다 null 확인
test_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [114]:
#null 데이터 제거
test_df.dropna(how='any', inplace=True)
test_df.shape

(49157, 3)

In [115]:
# 긍정 부정 레이블의 분포
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

## 2.텍스트 전처리

### train data set

In [116]:
# 한글과 공백이외는 제거
train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
train_df.head(3)

<ipython-input-116-463a1848189f>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [117]:
#한글 제외하고도 '' 데이터들 날려야한다 즉 영어로 적혀있더 데이터들이 널값이되어서 날리는거
train_df['document'].replace('', np.nan, inplace=True)
train_df.document.isnull().sum()

391

In [118]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145791, 3)

### train dataset

In [119]:
# 한글과 공백이외는 제거
test_df['document'] = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_df.head(3)

<ipython-input-119-397e579259ed>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test_df['document'] = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0


In [120]:
#한글 제외하고도 '' 데이터들 날려야한다 즉 영어로 적혀있더 데이터들이 널값이되어서 날리는거
test_df['document'].replace('', np.nan, inplace=True)
test_df.document.isnull().sum()

162

In [121]:
test_df.dropna(how='any', inplace=True)
test_df.shape

(48995, 3)

## 3. 한글처리

In [122]:
from konlpy.tag import Okt
okt =Okt()

In [123]:
# 한글 불용어 받아서 처리

In [124]:
df = pd.read_csv('data/한국어불용어100.txt', sep='\t', header=None)

In [125]:
df.head()

,0,1,2
0,이,VCP,0.018280
1,있,VA,0.011699
2,하,VV,0.009774
3,것,NNB,0.009733
4,들,XSN,0.006898


In [126]:
# 품사를 무시해서 중복이 삭제되었다
stopwords = set(df[0])
len(stopwords)

94

In [127]:
text = '교도소 이야기구먼 솔직히 재미는 없다 평점 조정'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [128]:
okt.morphs(text, stem=True) #동사를 원형으로 솔직히를 속직하다로 바꿈

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [129]:
' '.join(okt.morphs(text, stem=True)) #이렇게 합쳐서 countvect 에 넣을수 있다
#sting 만 읽을수 있다

'교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정'

In [130]:
from tqdm import tqdm_notebook
str_train = []
for sentence in tqdm_notebook(train_df.document): #tqdm이 뭐지
    morphs = okt.morphs(sentence.strip(), stem=True)
    #이제 stopwords 불용어 배제
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    
    str_train.append(temp_str)

<ipython-input-130-ff0e29c1dbf5>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sentence in tqdm_notebook(train_df.document): #tqdm이 뭐지


  0%|          | 0/145791 [00:00<?, ?it/s]

In [131]:
len(str_train)

145791

### test data

In [132]:
str_test = []
for sentence in tqdm_notebook(test_df.document): #tqdm이 뭐지
    morphs = okt.morphs(sentence.strip(), stem=True)
    #이제 stopwords 불용어 배제
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    
    str_test.append(temp_str)

<ipython-input-132-cbe94b66f224>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sentence in tqdm_notebook(test_df.document): #tqdm이 뭐지


  0%|          | 0/48995 [00:00<?, ?it/s]

In [133]:
len(str_test)

48995

In [134]:
y_train = train_df.label.values
y_test = test_df.label.values

## 4. Feature 변환

### CountVecotrizer

In [135]:
from sklearn.feature_extraction.text import CountVectorizer
#cvect = CountVectorizer(ngram_range=(1,2)) #시간없어서
cvect = CountVectorizer()

In [136]:
cvect.fit(str_train)
X_train = cvect.transform(str_train)
X_test = cvect.transform(str_test)
X_train.shape, X_test.shape

((145791, 42092), (48995, 42092))

## 5. 모델 학습 예측평가

In [137]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [138]:
lr = LogisticRegression(max_iter=500)
lr.fit(X_train, y_train) #학습

LogisticRegression(max_iter=500)

In [139]:
pred = lr.predict(X_test) #학습한 결과를 통해 test 예측

In [140]:
accuracy_score(y_test, pred)

0.8259006021022554

## 6. 실제 테스트

In [141]:
review1 = '처음에 황정민 실제 연기나 시상식 나오는 거부터 약간 반칙임..ㅎ 진짜에서 픽션으로 넘어갈 때도 다 믿어짐 황정민 황정민 하는거 이유가 있다'
review2 = ' 황정민 영화 중에 손꼽히는 실패작..'

In [142]:
review2.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')

' 황정민 영화 중에 손꼽히는 실패작..'

In [143]:
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review1)
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review2)
review1, review2

('처음에 황정민 실제 연기나 시상식 나오는 거부터 약간 반칙임ㅎ 진짜에서 픽션으로 넘어갈 때도 다 믿어짐 황정민 황정민 하는거 이유가 있다',
 ' 황정민 영화 중에 손꼽히는 실패작')

In [144]:
reviews = [review1, review2]
review_list = []
for review in reviews: #tqdm이 뭐지
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review)
    morphs = okt.morphs(review.strip(), stem=True)
    #이제 stopwords 불용어 배제
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    
    review_list.append(temp_str)

In [145]:
review_list

['처음 에 황정민 실제 연기 시상식 나오다 거 부터 약간 반칙 임 ㅎ 진짜 에서 픽션 으로 넘어가다 도 다 믿어지다 황정민 황정민 하다 이유 있다',
 '황정민 영화 에 손꼽다 실패 작']

In [146]:
review_cv = cvect.transform(review_list)
pred = lr.predict(review_cv)
pred

array([1, 0], dtype=int64)